In [3]:
import pandas as pd
import numpy as np
from models.utils import *
from deepctr.models import xDeepFM

In [4]:
path='./data/train_set_final.csv'                         #请把数据的路径写在这里
data=pd.read_csv(path,low_memory=False,encoding='utf-8')

In [5]:
data.columns 

Index(['广告id', '广告行业id', '广告账户id', '商品id', '商品类型', '素材尺寸', 'is_all_field',
       'has_product_id', 'time', 'age', 'area', 'behavior', 'connectionType',
       'consuptionAbility', 'device', 'education', 'gender', 'status', 'work',
       'price', 'crt_dateYear', 'crt_dateMonth', 'crt_dateWeek', 'crt_dateDay',
       'crt_dateDayofweek', 'crt_dateDayofyear', 'crt_dateIs_month_end',
       'crt_dateIs_month_start', 'crt_dateIs_quarter_end',
       'crt_dateIs_quarter_start', 'crt_dateIs_year_end',
       'crt_dateIs_year_start', 'crt_dateHour', 'crt_dateElapsed', '曝光量'],
      dtype='object')

In [ ]:
# new_columns=['sample_id','ad_id','size','ad_type_id','item_type','item_id','ad_account_id','price']
# data.columns[:,len(new_columns)]=new_columns

In [ ]:
sparse_features = ['广告id', '广告行业id', '广告账户id', '商品id', '商品类型','is_all_field',
       'has_product_id',] #在此加入稀疏特征名
multi_value_features=['素材尺寸','time', 'age', 'area','behavior','connectionType','area', ]#在此加入多值特征名
target = []#在此加入标签名
hashing=False
data=sparse_feature_encoding(data,sparse_features)
sequence_input,max_len_list=multi_value_feature_encoding(data,multi_value_features)
sparse_feat_list=sparse_feat_list_gen(data,sparse_feature,hashing)
sequence_output=sequence_feature_acquire(max_len_list,multi_value_features,hashing)

In [ ]:
# 3.generate input data for model
sparse_input = [data[feat.name].values for feat in sparse_feat_list]
dense_input = []#加入密集特征
sequence_input = [genres_list]
model_input = sparse_input + dense_input + \
              sequence_input  # make sure the order is right

In [ ]:
# 4.Define Model,compile and train
model = xDeepFM({"sparse": sparse_feat_list,
                "sequence": sequence_feature}, task='regression')

In [ ]:
#训练
model.compile("adam", "mse", metrics=['mse'], )
history = model.fit(model_input, data[target].values,
                    batch_size=256, epochs=10, verbose=2, validation_split=0.2, )

In [ ]:
#测试集测试
pred_ans = model.predict(test_model_input, batch_size=256)
print("test MSE", round(mean_squared_error(
    test[target].values, pred_ans), 4))

In [ ]:
#保存结果
result_for_submission=model.predict(test_model_input, batch_size=256)
print('Done!')
result_for_submission.to_csv(result_name,encoding='UTF-8')